In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import win32com.client
import getpass
import datetime
import pywintypes
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

# Function

In [ ]:
pd.set_option('display.width', None)
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)

def getDataToDF(xlws,start,end,first = False):
    '''
    input: excel worksheet path
    start: rows to begin
    end: rows to stop
    first: used for when doing getting data using bacthing
            true when the record is first wave
            false when the record is after first wave
    
    1. years that are more than the current year are replaced into 19xx. eg 2020 => 1920
    2. drop all dx date = 2020-01-01 (input errors)
    3. drop all rows where dx_Date is empty
    
    '''
    
    last_col = xlws.UsedRange.Columns.Count
    header = xlws.Range(xlws.Cells(1, 1), xlws.Cells(1, last_col)).Value
    content = xlws.Range(xlws.Cells(start, 1), xlws.Cells(end, last_col)).Value
    
    if first:
        data = list(content[1:])
    else:
        data = list(content[0:])
        
    for x in range(0,len(data)):
        data[x] = list(data[x])
        for y in range(0,len(data[x])):
            if isinstance(data[x][y], pywintypes.TimeType):
                temp = str(data[x][y]).rstrip("+00:00").strip()
                if temp[:2] == '20' and int(temp[:4]) >= int(now.year):
                    temp = '19' + temp[2:]
                data[x][y] = datetime.datetime.strptime(temp, "%Y-%m-%d")
    df = pd.DataFrame(data, columns=header[0])
    
    df.rename(columns={"dob_new":"dob","Age_new":"Age_@_Dx"}, inplace = True)
    
    df.fillna(value=pd.np.nan, inplace=True)
    
    #drop all rows where dx_date is empty
    df.drop(df[df["dx_date"].isnull()].index, inplace=True)

    return df

def processCol(df):                
            
    #for those dropdown variables in dictionary
    toDoDict = {}
    toDoDict['Gender'] = [('1','2','4','5','6','9997','9998','9999'),
                          ['Male','Female','Transsexual, NOS','Transsexual, natal male','Transsexual, natal female','NA',
                           'Others (intersex, disorders of sexual development/DSD)','Unknown']]
    toDoDict['c_tstage'] = [('1','2','3','4','5','6','7','8','9','11','12','13','41','42','43','44'),
                            ['T1','T2','T3','T4','Tx','T0','Tis','T1mic','unknown','T1a','T1b','T1c',\
                             'T4a','T4b','T4c','T4d']]
    toDoDict['cNstage'] = [('1','2','3','4','9','10','31','32','41','42','43'),
                           ['N0','N1','N2','N3','Nx','unknown','N2a','N2b','N3a','N3b','N3c']]
    toDoDict['cMstage'] = [('0','1','2','3','4','5','6','9997','9999'),
                           ['MX','M0','M1','M1a','M1b','M1c','M2','NA','Unknown']]     
    toDoDict['c_Staging'] = [('0','1','2','3','4','5','6','7','9','10','11','12','31','41'),
                             ['DCIS/LCIS non-invasive','Stage 1','Stage 2A','Stage 2B',\
                              'Stage 3A','Stage 3B','Stage 3C','Stage 4','Unknown','Stage 0',\
                              'Stage 1A','Stage 1B','Stage 2','Stage 3']] 
    toDoDict['tstage'] = [('1','2','3','4','5','6','7','8','9','11','12','13','41','42','43','44'),
                          ['T1','T2','T3','T4','Tx','T0','Tis','T1mic','unknown','T1a','T1b','T1c',\
                           'T4a','T4b','T4c','T4d']] 
    toDoDict['nstage'] = [('1','2','3','4','5','6','9','10','21','22','23','31','32','41','42','43'),
                          ['N0','N1','N2','N3','N1mic','N0 (i+)','Nx','unknown','N1a','N1b','N1c',\
                           'N2a','N2b','N3a','N3b','N3c']]
    toDoDict['Mstage'] = [('1','2','3','9'),('M0','M1','Mx','unknown')] 
    toDoDict['p_Staging'] = [('0','1','2','3','4','5','6','7','9','10','11','12','31','41'),
                             ['DCIS/LCIS non-invasive','Stage 1','Stage 2A','Stage 2B',\
                              'Stage 3A','Stage 3B','Stage 3C','Stage 4','Unknown','Stage 0',\
                              'Stage 1A','Stage 1B','Stage 2','Stage 3']]
    toDoDict['diff'] = [('0','1','2','3','9'),['grade 0','grade 1','grade 2','grade 3','unknown']] 
    toDoDict['TNM_Stage'] = [('0','1','2','3','4','5','6','7','9','10','11','12','13','31'),
                             ['DCIS/LCIS non-invasive','stage 1','stage 2A','stage 2B',\
                              'stage 3A','stage 3B','stage 3C','stage 4','Unknown','Stage 0',\
                              'stage 1A','stage 1B','Stage 3','stage 2']]
    toDoDict['ProgStage_AJCC8'] = [('0','4','11','12','21','22','31','32','33'),
                                   ['Stage 0','Stage IV','Stage IA','Stage IB','Stage IIA',\
                                   'Stage IIB','Stage IIIA','Stage IIIB','Stage IIIC']] 
    toDoDict['ER'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']] 
    toDoDict['PR'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']]
    toDoDict['cerbB2'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']]
    toDoDict['Her2'] = [('1','2','3','4','9'),['Positive','Negative','Not Done','Equivocal','Unknown']]
    toDoDict['cause_of_death'] = [('1','2','9'),['breast cancer related','N','unknown']]
    toDoDict['Count_as_DFS'] = [('0','1'),['N' ,'RECURRENCE (any)']]
    toDoDict['Count_as_OS'] = [('0','1'),['N' ,'Dead']]
    
    for k,v in toDoDict.items():
        '''
        1. convert numeric codes into category to standardise
            k: column name
            v[0]: numeric representation
            v[1]: category
        2. drop all those values that are outside of the defined numeric codes 
        '''
        
        v[1] = [x.lower() for x in v[1]]
        df[k].replace(to_replace =v[0],\
                      value = v[1],\
                      inplace = True)
        df[k] = df[k].str.lower()
        
        # allow null values
        v[1].append(pd.np.nan)
        
        # drop abnormal values   
        df.drop(df.loc[~df[k].isin(v[1])].index, inplace=True)
    
    DateList =  {"death_age":('death','dob')}
    for k,v in DateList.items():
        df[v[0]] = pd.to_datetime(df[v[0]])
        df[v[1]] = pd.to_datetime(df[v[1]])
        df[k] = (df[v[0]] - df[v[1]]).dt.days
        df[k] = df[k].floordiv(365.2425 , fill_value = pd.np.nan) 
        
        df = df.drop(columns=v[0])
        
        #drop negative age (input error)
        df.drop(df[df[k] < 0].index, inplace=True)
        
    #drop all rows where death age is lesser than age @ dx
    df.drop(df[df['death_age'] < df['Age_@_Dx']].index, inplace=True)
    
    return df

def dropColCDM(df, listToDrop):
    df = df.drop(columns=listToDrop)
    return df
        
def outToPickle(df,filename):
    with open(filename, 'wb') as f:
        pickle.dump(df, f)
    return None

def readPickle(filename):
    df = pickle.load(filename)
    return df

# Main Process

In [ ]:
FileToCheck = 'C:\\SMU_v2\\output.pkl'

if os.path.exists(FileToCheck):
    with open(FileToCheck, 'rb') as f:
        CDM = pickle.load(f)
else:
    # primary set up
    xlApp = win32com.client.Dispatch("Excel.Application")
    xlApp.Interactive = False
    xlApp.Visible = False

    # cwd = os.getcwd()
    cwd = "C:"
    path = str(cwd + "\\SMU_v2\\")
    files = os.listdir(path)

    # Put files into dataframe dict
    df_list = {}

    #require user input for password
    pwd = getpass.getpass('Enter file password: ')

    # Pick out 'xlsx' files:
    files_xls = [f for f in files if f[-4:] == 'xlsx']

    now = datetime.datetime.now()

    for f in files_xls:
        if "Clinical" in f:
            xlwb = xlApp.Workbooks.Open(path+f, False, True, None, pwd)
            xlws = xlwb.Worksheets(1) 
            last_row = xlws.UsedRange.Rows.Count

            CDM = getDataToDF(xlws,1,last_row)
            listToKeep = ['NRIC','dob','Gender','dx_date','c_tstage','cNstage','cMstage','c_Staging','tstage',\
                          'nstage','Mstage','p_Staging','diff','TNM_Stage','ProgStage_AJCC8','ER','PR',\
                          'cerbB2','Her2','size_precise','nodespos','cause_of_death','Date_for_DFS','Date_for_OS',\
                          'Count_as_DFS','Count_as_OS','Date_for_CSS','Count_as_CSS','death','Age_@_Dx']

            listToDrop = [i for i in CDM.columns if i not in listToKeep]

            CDM = dropColCDM(CDM,listToDrop)
            CDM = processCol(CDM)

            ##type casting to save space
            CDM.loc[:,"Gender"] = CDM["Gender"].astype("category")
            CDM.loc[:,"c_tstage"] = CDM["c_tstage"].astype("category")
            CDM.loc[:,"cNstage"] = CDM["cNstage"].astype("category")
            CDM.loc[:,"cMstage"] = CDM["cMstage"].astype("category")
            CDM.loc[:,"c_Staging"] = CDM["c_Staging"].astype("category")
            CDM.loc[:,"tstage"] = CDM["tstage"].astype("category")
            CDM.loc[:,"nstage"] = CDM["nstage"].astype("category")
            CDM.loc[:,"Mstage"] = CDM["Mstage"].astype("category")
            CDM.loc[:,"p_Staging"] = CDM["p_Staging"].astype("category")
            CDM.loc[:,"diff"] = CDM["diff"].astype("category")
            CDM.loc[:,"TNM_Stage"] = CDM["TNM_Stage"].astype("category")
            CDM.loc[:,"ProgStage_AJCC8"] = CDM["ProgStage_AJCC8"].astype("category")
            CDM.loc[:,"ER"] = CDM["ER"].astype("category")
            CDM.loc[:,"PR"] = CDM["PR"].astype("category")
            CDM.loc[:,"cerbB2"] = CDM["cerbB2"].astype("category")
            CDM.loc[:,"Her2"] = CDM["Her2"].astype("category")
            CDM.loc[:,"cause_of_death"] = CDM["cause_of_death"].astype("category")
            CDM.loc[:,"Count_as_DFS"] = CDM["Count_as_DFS"].astype("category")
            CDM.loc[:,"Count_as_OS"] = CDM["Count_as_OS"].astype("category")
            
            CDM['nodespos'].replace({'NA': pd.np.NaN}, inplace=True)
            
            CDM.loc[:,"size_precise"] = CDM["size_precise"].astype("float32")
            CDM.loc[:,"nodespos"] = CDM["nodespos"].astype("float16")
            
            outToPickle(CDM,FileToCheck )  

            #reset variables
            xlws = None
            xlwb.Close(False)
            xlwb = None

    #remove buffer and reset system settings
    xlApp.Interactive = True
    xlApp.Visible = True
    xlApp.Quit()
    xlApp = None

In [ ]:
CDM.to_csv("C:\\Users\\LINGXING\\Desktop\\clinical.csv",index=False)

# Bills

In [ ]:
# #for clinical data

# # primary set up
# xlApp = win32com.client.Dispatch("Excel.Application")
# xlApp.Interactive = False
# xlApp.Visible = False

# # cwd = os.getcwd()
# cwd = "C:"
# path = str(cwd + "\\SMU_v2\\")
# files = os.listdir(path)

# # Put files into dataframe dict
# df_list = {}

# #require user input for password
# pwd = getpass.getpass('Enter file password: ')

# # read in clinical data
# files_xls = ['Clinical Data_Masked_v2.xlsx']

# for f in files_xls:

#     #reading of data
#     xlwb = xlApp.Workbooks.Open(path+f, False, True, None, pwd)
#     xlws = xlwb.Worksheets(1) 
# #     last_row = 500
#     last_row = xlws.UsedRange.Rows.Count
#     last_col = xlws.UsedRange.Columns.Count
#     content = xlws.Range(xlws.Cells(1, 1), xlws.Cells(last_row, last_col)).Value
#     data = list(content[1:])
#     for x in range(0,len(data)):
#         data[x] = list(data[x])
#         for y in range(0,len(data[x])):
#             if isinstance(data[x][y], pywintypes.TimeType):
#                 temp = str(data[x][y]).rstrip("+00:00").strip()
#                 data[x][y] = datetime.datetime.strptime(temp, "%Y-%m-%d")
#     col_headers = content[0]
#     clinical = pd.DataFrame(data, columns=col_headers)
#     name = xlApp.ActiveWorkbook.Name
#     #reset variables
#     xlws = None
#     xlwb.Close(False)
#     xlwb = None
# df1 = clinical[clinical['dx_date'].isnull()][["NRIC","firstsee"]]
# df1.rename(columns={"firstsee":"bill_calculation_date"},inplace=True)
# df2 = clinical[clinical['dx_date'].notnull()][["NRIC","dx_date"]]
# df2.rename(columns={"dx_date":"bill_calculation_date"},inplace=True)
# frame = [df1,df2]
# processed_for_bills_df = pd.concat(frame)


In [ ]:

# FileToCheck = 'C:\\SMU_v2\\bills_output.pkl'

# bills_clean = pd.DataFrame()

# ##read in the service code mappings
# mappings = pd.read_excel("C:\\SMU_v2\\service code mapping_smu.xlsx")

# if False:#os.path.exists(FileToCheck):
#     bills_clean = pd.read_pickle(FileToCheck)
# else:
#     # primary set up
#     xlApp = win32com.client.Dispatch("Excel.Application")
#     xlApp.Interactive = False
#     xlApp.Visible = False

#     # cwd = os.getcwd()
#     cwd = "C:"
#     path = str(cwd + "\\SMU_v2\\")
#     files = os.listdir(path)

#     # Put files into dataframe dict
#     df_list = {}

#     #require user input for password
#     pwd = getpass.getpass('Enter file password: ')

#     # Pick out 'xlsx' files:
#     files_xls = ['Bills Data_10-12k (MASKED)v2.xlsx', 'Bills Data_12-14k (MASKED)v2.xlsx', 'Bills Data_14-16k (MASKED)v2.xlsx', 'Bills Data_16-18k (MASKED)v2.xlsx', 'Bills Data_18-20k (MASKED)v2.xlsx', 'Bills Data_1st 2k (MASKED)v2.xlsx', 'Bills Data_2-4k (MASKED)v2.xlsx', 'Bills Data_20-22k (MASKED)v2.xlsx', 'Bills Data_22-24k (MASKED)v2.xlsx', 'Bills Data_24-26k (MASKED)v2.xlsx', 'Bills Data_4-6k (MASKED)v2.xlsx', 'Bills Data_6-8k (MASKED)v2.xlsx', 'Bills Data_8-10k (MASKED)v2.xlsx', 'Bills Data_last 1k (MASKED)v2.xlsx']

#     for f in files_xls:
        
#         #reading of data
#         first = True
#         counter = 1
#         xlwb = xlApp.Workbooks.Open(path+f, False, True, None, pwd)
#         xlws = xlwb.Worksheets(1) 
# #         last_row = 500
#         last_row = xlws.UsedRange.Rows.Count
#         last_col = xlws.UsedRange.Columns.Count
#         n=50000
#         while counter < last_row:
#             print("Processing {}".format(counter))
#             prev_counter = counter
#             counter = min(counter+n,last_row)
#             content = xlws.Range(xlws.Cells(prev_counter, 1), xlws.Cells(counter, last_col)).Value
#             if first:
#                 print(last_row)
#                 first = False
#                 col_headers = content[0]
#                 data = list(content[1:])
#             else:
#                 data = list(content[0:])
#             for x in range(0,len(data)):
#                 data[x] = list(data[x])
#                 for y in range(0,len(data[x])):
#                     if isinstance(data[x][y], pywintypes.TimeType):
#                         temp = str(data[x][y]).rstrip("+00:00").strip()
#                         data[x][y] = datetime.datetime.strptime(temp, "%Y-%m-%d")

#             bills = pd.DataFrame(data, columns=col_headers)
#             #cleaning of data
#             ##remove all bills with gross cost of NA
#             bills['Gross..exclude.GST.'].replace("NA",np.nan,inplace=True)
#             bills = bills.dropna(subset=['Gross..exclude.GST.'])

#             ##removal of several unused columns
#             unused = ["Gender","Date.of.Birth","Service.Description","Net..exclude.GST.","Net.Payable","Service.Cost",
#                      "Billed.Qty","Service.Entered.Price","Doctor.Surcharge..SVC.","Total.Cost","TOSP","Billing.Category.Description",
#                      "Billing.Status.Description","Billing.Date","Admit.Sub.Specialty.Description","Admit.Specialty.Description",
#                      "Admit.Accommodation.Category.Description","Diagnosis.Description..ICD10.","Diagnosis.Description"]
#             bills = bills.drop(unused,axis = 1)

#             ##can consider removing these columns 
#             unused = ["Service.Category.1.Code", "Service.Category.1.Description","Service.Category.2.Code","Service.Category.2.Description",
#                      "Service.Summary.Code"]
#             bills = bills.drop(unused,axis = 1)

#             ##replace all expected unknown with np.nan
#             bills = bills.replace("Expected Unknown",np.nan)
#             bills = bills.replace("EXPUNKNOWN",np.nan)

#             ##type casting to save space
#             bills.loc[~bills["Surgery.Date"].isna(),"Surgery.Date"] = True
#             bills.loc[bills["Surgery.Date"].isna(),"Surgery.Date"] = False
#             ##add the processed bills to a clean df
            
#             bills_clean = bills_clean.append(bills.reset_index())
            
#         name = xlApp.ActiveWorkbook.Name
#         #reset variables
#         xlws = None
#         xlwb.Close(False)
#         xlwb = None
        
#         ##type casting to save space
#         bills_clean.loc[:,"Gross..exclude.GST."] = bills_clean["Gross..exclude.GST."].astype("uint64")
#         bills_clean.loc[:,"Service.Qty"] = bills_clean["Service.Qty"].astype("uint16")
#         bills_clean.loc[:,"Institution.Code"] = bills_clean["Institution.Code"].astype('category')
#         bills_clean.loc[:,"Service.Summary..Description"] = bills_clean["Service.Summary..Description"].astype('category')
#         bills_clean.loc[:,"Service.Code"] = bills_clean["Service.Code"].astype('category')
#         bills_clean.loc[:,"Service.Short.Text"] = bills_clean["Service.Short.Text"].astype('category')
#         bills_clean.loc[:,"Service.Department.Description"] = bills_clean["Service.Department.Description"].astype('category')
#         bills_clean.loc[:,"Diagnosis.Code"] = bills_clean["Diagnosis.Code"].astype('category')
#         bills_clean.loc[:,"Diagnosis.Code..ICD10."] = bills_clean["Diagnosis.Code..ICD10."].astype('category')

#     #remove buffer and reset system settings
#     xlApp.Interactive = True
#     xlApp.Visible = True
#     xlApp.Quit()
#     xlApp = None

#     bills_clean = bills_clean.drop(["index"],axis = 1)
    
#     print("Done")
    
#     ##save the df
#     outToPickle(bills_clean,FileToCheck)

In [ ]:
bills_clean = pd.read_pickle('C:\\SMU_v2\\bills_output_datefiltered.pkl')
# MONTH = 30
# YEAR = 365
# print("Before checking for dates")
# print(mem_usage(bills_clean))
# FileToCheck = 'C:\\SMU_v2\\bills_output_datefiltered.pkl'
# unique = []
# bills_clean.reset_index(inplace=True)
# for i,j in processed_for_bills_df.iterrows():
#     if j["NRIC"] in unique:
#         pass
#     else:
#         unique.append(j["NRIC"])
#         #take out all bills that are related to the nric
#         subbills = bills_clean.loc[bills_clean["Patient.ID"] == j["NRIC"]]
#         difference = subbills['Service.Date.From.Date'] - j["bill_calculation_date"]
#         condition1 = difference.astype("timedelta64[D]") < (-6*MONTH) 
#         condition2 = difference.astype("timedelta64[D]") > (10*YEAR) 
#         bills_clean = bills_clean.drop(subbills[condition1 | condition2].index)
#         print(j['NRIC'],bills_clean.shape[0])
# print("After checking dates")
# print(mem_usage(bills_clean))
# print("done")
# outToPickle(bills_clean,FileToCheck)

In [ ]:
MONTH = 30
YEAR = 365

#calculate memory usage
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

def get_patient_bills(patient_id):
    """
    input: patient id (str)
    output: A dataframe containing all bills of given patient
    """
    subset = bills_clean[bills_clean["Patient.ID"] == patient_id]
    return subset

def get_cost_runningtotal(date,patient_bills):
    """
    input: date(pd.Timestamp) and dataframe of patient's bills (1 patient only)
    output: from date, calculate the sum of bills [6 months before, 6 months after, 1 year after, 2 years after, 5 years after, 10 years after]
    """
    try:
        difference = patient_bills['Service.Date.From.Date'] - date
        
        condition1 = difference.astype("timedelta64[D]") >= (-6*MONTH)
        condition2 = difference.astype("timedelta64[D]") >= 0 
        condition3 = difference.astype("timedelta64[D]") >= (6*MONTH) 
        condition4 = difference.astype("timedelta64[D]") >= (1*YEAR) 
        condition5 = difference.astype("timedelta64[D]") >= (2*YEAR) 
        condition6 = difference.astype("timedelta64[D]") >= (5*YEAR) 
        condition7 = difference.astype("timedelta64[D]") >= (10*YEAR) 
        before_6m = patient_bills[condition1 & (condition2==False)]['Gross..exclude.GST.'].sum()
        after_6m = patient_bills[condition2 & (condition3==False)]['Gross..exclude.GST.'].sum()
        after_1y = patient_bills[condition2 & (condition4==False)]['Gross..exclude.GST.'].sum()
        after_2y = patient_bills[condition2 & (condition5==False)]['Gross..exclude.GST.'].sum()
        after_5y = patient_bills[condition2 & (condition6==False)]['Gross..exclude.GST.'].sum()
        after_10y = patient_bills[condition2 & (condition7==False)]['Gross..exclude.GST.'].sum()
        return [before_6m, after_6m, after_1y, after_2y, after_5y, after_10y]
    except:
        return [0,0,0,0,0,0]
    
def get_cost_timeperiod(date,patient_bills):
    """
    input: date(pd.Timestamp) and dataframe of patient's bills (1 patient only)
    output: from date, calculate the sum of bills [6 months before, 6 months after, 1 year after, 2 years after, 5 years after, 10 years after]
    """
    try:
        difference = patient_bills['Service.Date.From.Date'] - date
        condition1 = difference.astype("timedelta64[D]") > (-6*MONTH)
        condition2 = difference.astype("timedelta64[D]") >= 0 
        condition3 = difference.astype("timedelta64[D]") >= (6*MONTH) 
        condition4 = difference.astype("timedelta64[D]") >= (1*YEAR) 
        condition5 = difference.astype("timedelta64[D]") >= (2*YEAR) 
        condition6 = difference.astype("timedelta64[D]") >= (5*YEAR) 
        condition7 = difference.astype("timedelta64[D]") >= (10*YEAR) 
        before_6m = patient_bills[condition1 & (condition2 == False)]['Gross..exclude.GST.'].sum()
        after_6m = patient_bills[condition2 & (condition3 == False)]['Gross..exclude.GST.'].sum()
        after_1y = patient_bills[condition3 & (condition4 == False)]['Gross..exclude.GST.'].sum()
        after_2y = patient_bills[condition4 & (condition5 == False)]['Gross..exclude.GST.'].sum()
        after_5y = patient_bills[condition5 & (condition6 == False)]['Gross..exclude.GST.'].sum()
        after_10y = patient_bills[condition6 & (condition7 == False)]['Gross..exclude.GST.'].sum()
        return [before_6m, after_6m, after_1y, after_2y, after_5y, after_10y]
    except:
        return [0,0,0,0,0,0]

In [ ]:
prices = []
counter = 1
for i,j in CDM.iterrows():
    counter+=1
    prices.append(
        get_cost_timeperiod(j["dx_date"],
                 get_patient_bills(j["NRIC"])))
    if counter%10 == 0:
        print(counter)
        
test = pd.DataFrame(prices,columns=["6 months before", "6 months after", "1 year after", "2 year after", "5 years after", "10 years after"])
test.to_pickle("C:\\SMU_v2\\price_timeperiod.pkl")